In [6]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from pandas.core import datetools

In [18]:
def read_dataset(path):
    data = pd.read_csv(path)
    return data

In [19]:
senate_name_dict = {}
house_name_dict = {}
name_dict = {}
def enumerate_districts(df, type=None):
    names = df['District']
    count = 0
    for name in names:
        if type == "Senate":
            if name not in senate_name_dict:
                senate_name_dict[name] = count
                count += 1
        elif type == "House":
            if name not in house_name_dict:
                house_name_dict[name] = count
                count += 1
        else:
            if name not in name_dict:
                name_dict[name] = count
                count += 1

In [20]:
def replace_district(x, type=None):
    if type == "Senate":
        return senate_name_dict[x]
    elif type == "House":
        return house_name_dict[x]
    else:
        return name_dict[x]

In [152]:
# this will be different than historical_model
def clean_data(df, type=None):
    # make all text binary
    df = df.drop("name", axis=1)
    df = df.drop("Unnamed: 0", axis=1)
    df['sex'].replace('f', 1, inplace=True)
    df['sex'].replace('m', 0, inplace=True)
    df['party'].replace('Democratic', 1, inplace=True)
    df['party'].replace('Republican', 0, inplace=True)
    
    # fill NaN's with mean from column
    df['sex'] = df['sex'].fillna(round(df['sex'].mean()))
    df['party'] = df['party'].fillna(df['party'].mean())
    df['Amount'] = df['Amount'].fillna(df['Amount'].mean())
    
    for col in df.columns:
        if col == "vote_count":
            continue
        if df[col].dtype != float and df[col].dtype != int:
            if "," in list(df[col])[0]:
                df[col] = df[col].apply(lambda x: str(x).replace(",", "").replace('nan', 'NaN')).astype(float)
    
    df['vote_count'] = df['vote_count'].apply(lambda x: str(x).replace(",", "").replace('nan', 'NaN')).astype(float)
    df['vote_count'] = df['vote_count'].fillna(df['vote_count'].mean())
    df['vote_percent'] = df['vote_percent'].fillna(df['vote_percent'].mean())
    df['District'] = df["District"].apply(lambda x: replace_district(x, type))
    
    return df

In [153]:
def independent_columns(A, tol =0): #= 1e-05):
    Q, R = np.linalg.qr(A)
    independent = np.where(np.abs(R.diagonal()) > tol)[0]
    
    return independent

In [154]:
def classify_ols(train_df, val_df):
    y_train = train_df['sex']
    X_train = train_df.drop('sex', axis=1)
    
    y_val = val_df['sex']
    X_val = val_df.drop('sex', axis=1)
    
    independent = independent_columns(X_train)
    X_train = X_train.iloc[:, independent]
    X_val = X_val.iloc[:, independent]
    print("Rank is {}".format(X_train.shape[1]))
    X_train.to_csv("X_train.csv")

    classifier = sm.Logit(y_train, X_train)
    results = classifier.fit(method='ncg')
    pred = round(results.predict(X_val))
    return accuracy_score(y_val, pred)

In [155]:
def classify_glm(train_df, val_df):
    y_train = train_df['sex']
    X_train = train_df.drop('sex', axis=1)
    
    y_val = val_df['sex']
    X_val = val_df.drop('sex', axis=1)
    
    independent = independent_columns(X_train)
    X_train = X_train.iloc[:, independent]
    X_val = X_val.iloc[:, independent]
    print("Rank is {}".format(X_train.shape[1]))

    classifier = sm.GLM(y_train, X_train)
    results = classifier.fit()
    print(results.summary())
    pred = round(results.predict(X_val))
    return accuracy_score(y_val, pred)

In [156]:
senate_data = read_dataset("VoteBuilder_and_all_data_Senate.csv")
enumerate_districts(senate_data, "Senate")
    
cleaned_senate = clean_data(senate_data,"Senate")
print(cleaned_senate)

house_data = read_dataset("VoteBuilder_and_all_data_House.csv")
enumerate_districts(house_data, "House")
    
cleaned_house = clean_data(house_data,"House")
print(cleaned_house)

# print("OLS Logistic Regression: ")
# print(classify_ols(cleaned_train_senate, cleaned_val_senate))
# print("OLS Generalized Linear Model: ")
# print(classify_glm(cleaned_train_senate, cleaned_val_senate))

object
District
Dem_0_of_4
Dem_1_of_4
Dem_2_of_4
Dem_3_of_4
Dem_4_of_4
Rep_0_of_4
Rep_1_of_4
Rep_2_of_4
Rep_3_of_4
Rep_4_of_4
UnEn_0_of_4
UnEn_1_of_4
UnEn_2_of_4
UnEn_3_of_4
UnEn_4_of_4
Dem_Caucasian_high
Dem_Caucasian_med
Dem_Caucasian_low
Dem_Hispanic_high
Dem_Hispanic_low
Rep_Caucasian_high
Rep_Caucasian_low
UnEn_Asian_high
UnEn_Caucasian_high
UnEn_Caucasian_med
UnEn_Caucasian_low
UnEn_Hispanic_high
UnEn_Hispanic_low
UnEn_unknown_race
Unkn_Caucasian_high
Dem_Male
Dem_Female
Dem_Male.1
Dem_Female.1
UnEn_Male
UnEn_Female
Total_Voters
vote_count
    District  Dem_0_of_4  Dem_0_of_4_%  Dem_1_of_4  Dem_1_of_4_%  Dem_2_of_4  \
0          0      2009.0          2.98      2723.0          4.03      2217.0   
1          1       406.0          3.13       486.0          3.75       406.0   
2          2      2493.0          3.09      3349.0          4.15      2704.0   
3          3      1178.0          2.78      1940.0          4.57      1603.0   
4          4        77.0          1.59       133

TypeError: argument of type 'float' is not iterable